# Preporuka vesti

#### Importovanje

In [6]:
import tensorflow.compat.v1.keras as keras
from tensorflow.compat.v1.keras import layers
from att_layer import AttLayer2
from overwrite_masking import OverwriteMasking
from compute_masking import ComputeMasking

#### Kreiranje enkodera vesti

In [ ]:
def build_news_encoder(hparams, embedding_layer, seed):
    # Parametri pronadjeni na mind dokumentaciji!
    filter = 400
    window = 3
    padding = "same" # Sto znaci da ce se pad sastojati od 0 podjednako sa svih strana
    title_size = 30
    dropout = 0.2

    sequences_input_title = keras.Input(shape=(title_size,), dtype="int32")
    embedded_sequences_title = embedding_layer(sequences_input_title)

    y = layers.Dropout(dropout)(embedded_sequences_title)
    y = layers.Conv1D(
        filter,
        window,
        activation='relu',
        padding=padding,
        bias_initializer=keras.initializers.Zeros(),
        kernel_initializer=keras.initializers.glorot_uniform(seed=seed),
    )(y)
    y = layers.Dropout(dropout)(y)
    y = layers.Masking()(
        OverwriteMasking()([y, ComputeMasking()(sequences_input_title)])
    )
    pred_title = AttLayer2(hparams.attention_hidden_dim, seed=seed)(y)
    model = keras.Model(sequences_input_title, pred_title, name="news_encoder")
    return model

In [ ]:
def build_lstur(hparams, word2vec_embedding):
    """The main function to create LSTUR's logic. The core of LSTUR
    is a user encoder and a news encoder.
    Returns:
        object: a model used to train.
        object: a model used to evaluate and inference.
    """

    his_input_title = keras.Input(
        shape=(hparams.his_size, hparams.title_size), dtype="int32"
    )
    pred_input_title = keras.Input(
        shape=(hparams.npratio + 1, hparams.title_size), dtype="int32"
    )
    pred_input_title_one = keras.Input(
        shape=(
            1,
            hparams.title_size,
        ),
        dtype="int32",
    )
    pred_title_reshape = layers.Reshape((hparams.title_size,))(pred_input_title_one)
    user_indexes = keras.Input(shape=(1,), dtype="int32")

    embedding_layer = layers.Embedding(
        word2vec_embedding.shape[0],
        hparams.word_emb_dim,
        weights=[word2vec_embedding],
        trainable=True,
    )

    titleencoder = build_news_encoder(embedding_layer)

